# 🔡 Step 05: OpenAI Embedding으로 텍스트 벡터화

이 노트북에서는 이전 단계에서 전처리한 문서들을 OpenAI의 Embedding API를 사용하여 벡터화합니다.

## 📝 주요 내용
1. OpenAI API 설정
2. 텍스트 임베딩 함수 구현
3. 문서 벡터화 및 저장
4. 벡터 품질 확인

## 1️⃣ 필요한 라이브러리 임포트

텍스트 벡터화에 필요한 라이브러리들을 임포트합니다.

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from typing import List, Dict
from dotenv import load_dotenv
from openai import OpenAI
from tqdm.notebook import tqdm

# .env 파일에서 환경변수를 로드합니다
load_dotenv()

# OpenAI 클라이언트를 초기화합니다
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

## 2️⃣ 이전 단계의 전처리된 문서 로드

Step 04에서 저장한 전처리된 문서들을 로드합니다.

In [ ]:
# 전처리된 문서를 로드합니다
with open('../data/processed/documents.json', 'r', encoding='utf-8') as f:
    documents = json.load(f)

print(f"로드된 문서 수: {len(documents)}")
print("\n문서 예시:")
print(json.dumps(documents[0], indent=2, ensure_ascii=False))

## 3️⃣ 텍스트 임베딩 함수 구현

OpenAI의 text-embedding-ada-002 모델을 사용하여 텍스트를 벡터화하는 함수를 구현합니다.

In [ ]:
def get_embedding(text: str) -> List[float]:
    """
    OpenAI API를 사용하여 텍스트의 임베딩을 생성합니다.
    
    Args:
        text (str): 임베딩할 텍스트
        
    Returns:
        List[float]: 1536차원의 임베딩 벡터
    """
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response.data[0].embedding

def batch_get_embeddings(texts: List[str], batch_size: int = 100) -> List[List[float]]:
    """
    여러 텍스트의 임베딩을 배치 처리합니다.
    
    Args:
        texts (List[str]): 임베딩할 텍스트 리스트
        batch_size (int): 배치 크기
        
    Returns:
        List[List[float]]: 임베딩 벡터 리스트
    """
    embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="임베딩 생성 중"):
        batch = texts[i:i + batch_size]
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=batch
        )
        batch_embeddings = [data.embedding for data in response.data]
        embeddings.extend(batch_embeddings)
        
    return embeddings

## 4️⃣ 문서 벡터화

전체 문서를 벡터화하고 결과를 저장합니다.

In [ ]:
# 문서 텍스트 추출
texts = [doc['content'] for doc in documents]

# 임베딩 생성
embeddings = batch_get_embeddings(texts)

# 임베딩을 문서에 추가
for doc, embedding in zip(documents, embeddings):
    doc['embedding'] = embedding

# 벡터화된 문서 저장
output_path = '../data/processed/documents_with_embeddings.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(documents, f, ensure_ascii=False, indent=2)

print(f"벡터화된 문서가 {output_path}에 저장되었습니다.")

## 5️⃣ 벡터 품질 확인

생성된 임베딩의 품질을 간단히 확인합니다.

In [ ]:
def cosine_similarity(v1: List[float], v2: List[float]) -> float:
    """
    두 벡터 간의 코사인 유사도를 계산합니다.
    """
    dot_product = np.dot(v1, v2)
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    return dot_product / (norm1 * norm2)

# 샘플 문서 선택
sample_doc = documents[0]
sample_embedding = sample_doc['embedding']

# 다른 문서들과의 유사도 계산
similarities = []
for doc in documents[1:10]:  # 처음 10개 문서만 비교
    sim = cosine_similarity(sample_embedding, doc['embedding'])
    similarities.append({
        'content': doc['content'][:100] + '...',  # 내용 일부만 표시
        'similarity': sim
    })

# 유사도 결과 출력
print("기준 문서:")
print(sample_doc['content'][:100] + '...\n')

print("유사도가 높은 문서들:")
for item in sorted(similarities, key=lambda x: x['similarity'], reverse=True)[:3]:
    print(f"\n유사도: {item['similarity']:.4f}")
    print(f"내용: {item['content']}")

## 🎯 정리

1. OpenAI의 text-embedding-ada-002 모델을 사용하여 문서를 벡터화했습니다.
2. 배치 처리를 통해 효율적으로 임베딩을 생성했습니다.
3. 코사인 유사도를 통해 벡터의 품질을 확인했습니다.

다음 단계에서는 이 벡터화된 문서들을 Elasticsearch에 저장하고 검색 시스템을 구축해보겠습니다.